In [ ]:
from torchvision.models import resnet18
import torch
import sys

from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from pycocotools.coco import COCO


In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
def get_model(out_features = 1, mode = 'tune', parent = 'pretrained', randomize = False):
    # Load the model
    model = resnet18(pretrained = (parent == 'pretrained'))
    # Change the classification layer
    model.fc = torch.nn.Linear(in_features = 512, out_features = out_features)
    # Load the in the parent model weights
    if parent != 'pretrained':
        model.load_state_dict(torch.load(parent, map_location=torch.device('cpu')))
    if randomize:
        model.fc = torch.nn.Linear(in_features = 512, out_features = out_features)
    # Setup the trainable parameters
    if mode == 'tune':
        return model, model.parameters()
    elif mode == 'transfer':
        for param in model.parameters():
            param.requires_grad = False
        model.fc.weight.requires_grad = True
        model.fc.bias.requires_grad = True
        return model, model.fc.parameters()
    elif mode == 'eval':
        for param in model.parameters():
            param.requires_grad = False
        model.eval()
        return model
    else:
        print('ResNet.py: Could not determine trainable parameters')
        sys.exit(0)

In [ ]:
m = get_model(out_features=80, mode = 'eval', parent='../../../model.pt')

In [ ]:
img = Image.open("/Users/acabrera/dev/data/coco-2017/val2017/000000174482.jpg")

In [ ]:
img.show()

In [ ]:
img_t = preprocess(img)

In [ ]:
o = m(img_t.unsqueeze(0)).numpy()

In [ ]:
o.squeeze()

In [ ]:
np.argwhere(o > 0)

In [ ]:
coco = COCO("/Users/acabrera/dev/data/coco-2017/instances_val2017.json")

In [ ]:
cats = coco.loadCats(coco.getCatIds())
indices = [int(x['id']) for x in cats]
names = [x['name'].replace(' ', '+') for x in cats]
name2index = {}
for i, v in enumerate(names):
	name2index[v] = i
index2name = list(name2index)

In [ ]:
print(index2name)

In [ ]:
imgs = coco.loadImgs(coco.getImgIds())
imgs

In [ ]:
df = pd.DataFrame.from_dict(imgs)
df.set_index('id', inplace=True)
for col in index2name:
	df[col] = 0
df.head()

In [ ]:
for img in imgs:
	anns = coco.loadAnns(coco.getAnnIds(img['id'], iscrowd = None))
	label = np.zeros((91))  # Each 'label' vector is large enough for easy indexing, but this means it contains unused indices
	for ann in anns:
		label[ann['category_id']] = 1.0
	label = label[indices] # Remove the unused indices
	df.at[img['id'], 'label'] = '['
	for i, lab in enumerate(label):
		if lab > 0:
			df.at[img['id'], 'label'] = df.at[img['id'], 'label'] + index2name[i] + ','
			df.at[img['id'], index2name[i]] = df.at[img['id'], index2name[i]] + 1
	df.at[img['id'], 'label'] = df.at[img['id'], 'label'] + ']'	
	df.at[img['id'], 'nplabel'] = label.tobytes()

In [ ]:
df.head()

In [ ]:
df.to_csv('coco-tagging.csv')